# Lab 6 - Visually analyze iPhone app downloads
*© 2021 Colin Conrad*

Welcome to Week 6 of INFO 6270! Last week we explored dataframes. In this lab we will build on those concepts by exploring dataframe visualization methods. Python comes with a few good visualization tools, though (as we will see) they are not a replacement for a good data visualization tool such as Tableau. It would be interesting to have a discussion about the strengths and weaknesses of the tools. 

The default visualization library that is leveraged by Pandas is called [Matplotlib](https://matplotlib.org/). This library makes it super-fast to create attractive (and publishable) visualizations, and is very flexible. However, this library can also be annoying to get it to do exactly what you want. However, when used with Pandas, some of the more difficult tasks are simplified. In this lab you will visualize a different data frame, and work through many of Pandas' visualization features. We will then finish the lab by exploring [Seaborn](https://seaborn.pydata.org/introduction.html), a newer Python visualization library with many beautiful visualization features.

**This week, we will achieve the following objectives:**
- Visualize a dataframe in Python
- Visualize a grouped dataframe
- Change your plot styles
- Visualize data using Seaborn

Weekly reading: Visualization. [_Pandas User Guide_](https://pandas.pydata.org/pandas-docs/stable/user_guide/visualization.html)

# Case: Apple Appstore
Pretty much everyone knows about the Apple iPhone; after all, this was *the* defining smartphone. However, iPhone's reign as the big boss of smartphones may be coming to an end. According to Gartner, as Q3 2019 [iPhone has seen its year-over-year sales decline by 10\%](https://www.gartner.com/en/newsroom/press-releases/2019-11-26-gartner-says-global-smartphone-demand-was-weak-in-thi). This is due to increasing competition from some of its android competitors.

Part of Apple's key to success has been the iPhone App Store. Unlike its Android counterparts, all Apple apps are developed by licensed developers and carefully screened for malware. To analyze other factors in its success, we can observe data from the Appstore itself. Using this dataset provided by [Ramanathan on Kaggle](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps) we can visualize features of the dataset which might have contributed to the success of the appstore.

# Objective 1: Visualize a dataframe in Python
The first thing we will do is create some basic visualizations. Pandas has some [great documentation on visualization](https://pandas.pydata.org/pandas-docs/stable/user_guide/visualization.html#basic-plotting-plot) which I strongly encourage you to read. This documentation provides more details than I can give in this exercise, though I will make an effort to highlight some key points.

Let's start by importing the Pandas dataframe, as before. We will similarly import `numpy`, but also a different library called `matplotlib`. This tool is a plotting library which is designed to integrate with pandas. As before, we will import a csv file, this time from Apple. A word of caution: this data is moderately messy, though is, again, a real life example.

In [ ]:
import pandas as pd # import pandas 
import numpy as np # imports numpy, we need this for Seaborn later
import matplotlib.pyplot as plt # notation for importing matplotlib

apps = pd.read_csv('data/w6_apple.csv') # command pandas to import the data

### Dataframe head (again!)
Most of the time, visualization tutorials start with easy datasets. We will not do this, so that we can be closer to a real-world scenario. Most of the data that you will work with in life (or even in your final projects) are not well-tailored to visualization. Fortunately, we have Pandas and our Python programming skills to help.

I recommend always starting by understanding the data. In this case, we have a series of iPhone apps with a few interesting fields. Here are some details on ones which are potentially interesting and non-obvious to me:
- **rating_count_tot**: Total number of ratings for all versions of the app
- **rating_count_ver**: Number of ratings for this version of the app
- **sup_devisces.num**: Number of Apple devices that the app supports
- **lang.num**: The number of (human) languages which the app supports

In [ ]:
apps.head()

### Visualizing with pandas.plot()
Pandas really is designed for data scientists. In addition to the dataframe features which we observed last week, pandas also comes pre-built with plotting features. Pandas also provides some [excellent documentation on plotting](https://pandas.pydata.org/pandas-docs/stable/user_guide/visualization.html#basic-plotting-plot) which I encourage you to read.

Let's start by trying to plot our dataframe. Surprisingly, pandas is smart enough to allow us to do this, though it is certainly not pretty! We will need to be a bit more specific about what we want to visualize before proceeding.

In [ ]:
apps.plot() # plots the dataframe; completely unreadable

The graph above is meaningless. Let's try to focus on more specific elements of this dataframe.

### Plot a series
The first way that we might make sense of the data is by visualizing series from the data. However, simply using `.plot()` with a series suffers from serious problems. Try executing the visualization below, which attempts to visualize the user ratings for the current version of each app. **This is is not yet intelligible.**

In [ ]:
apps['user_rating_ver'].plot() # visualize user ratings for the current version of the app

Though pandas is smart, it is not **that** smart. When we naively try to visualize the version, we end up with unreadable garbage. Fortunately, we can alter our series to suit our needs. Consider visualizing the sorted user ratings. Note that we have to explicitly tell pandas to not order these by the original index values.

In [ ]:
apps['user_rating_ver'].sort_values().plot(use_index=False) # sort the series and then plot it

This is looking much better! However, we can do a lot more to refine the data and the corresponding visualization.
### Specifying axes and figure sizes
It's important to remember that when making plots we specify an axis. For instance, if we only wanted to show the number of ratings received for an app, we could specify the y axis as `rating_count_tot`. To make our graph more readable, we can also change the figure size by specifying `figsize=(12,6)` (12 inches by 6 inches).

In [ ]:
apps.plot(y='rating_count_tot', figsize=(12,6))

If we wished instead to visualize a subset of the data, say only highly rated apps, we could create a subset similarly to Lab 8. We can then use `plot()` to visualize the results. The result will still not be informative, but it is progress.

In [ ]:
highly_rated = apps[(apps['user_rating'] == 5) & # a subset of apps with ratings of 5 and at least 100 000 ratings
                    (apps['rating_count_tot'] > 100000)] # It will return only 17 values

highly_rated.plot(y='rating_count_tot') # manually specify the y value, in this case rating count

### Try a different plot
From here, our visualization needs two things in order to be useful. The first is a useful visualization for comparing the different states identified. The second is a decent x label. We can solve these problems by asking pandas to create a bar plot with the `track_name` axis. As you recall, track name is the name of the app. 

**Note:** you will probably get a warning about a missing glyph. This is because of encoding issues with some of the characters.

In [ ]:
sorted_rated = highly_rated.sort_values(by='rating_count_tot') # sort the data

sorted_rated.plot.bar(x='track_name', y='rating_count_tot', figsize=(12,6)) # specify bar plot with the x value of track name

This is a fine graph of the apps with an average rating of 5 and at large number of ratings! I am sure that many of you use these apps (I counted 3 which I use). 

### Remove outliers
Finally, there are other types of visualizations which could be useful. For instance, a scatter plot can be used to compare the variance between two variables. Let's plot the rating count and price to see whether there is a relationship.

In [ ]:
price_rating = apps[['price', 'rating_count_tot', 'rating_count_ver']] # we will include three series in this subset dataframe

price_rating.plot.scatter(x='price', y='rating_count_tot', figsize=(12,6)) # we choose to only visualize two of them

Right away, it is clear that there are some outliers with a small number of ratings and high price, as well as a high number of ratings and low price. We can remove the outliers to try and make sense of this graph. There are many ways to remove outliers, such as by using standard deviation. However, for the purposes of *Introduction to Data Science* it is sufficient to simply remove values that seem too extreme. The code below removes price values which are greater than 10 and rating counts which are greater than 10000. 

In [ ]:
apps_clean = apps[(apps['price'] < 25) & # maximum price $25
                  (apps['rating_count_tot'] < 100000)] # max total ratings 10 000

apps_clean.plot.scatter(x='price', y='rating_count_tot', figsize=(12,6))

## *Challenge Question 1 (1 point)*
Take a subset of the data where `prime_genere` is equal to `Games`. Generate a scatter plot with the number of languages on one axis and the price on the other. Do you think this tells us something about the relationship between these variables? 

In [ ]:
# insert your code here

# Objective 2: Visualize a grouped dataframe
So far so good. What we have done so far works for continuous variables such as `price`, but not nominal variables such `prime_genre`. For instance, if we simply visualize `prime_genre` on the x axis, we will get a nonsensical graph. 

In [ ]:
apps.plot(x='prime_genre', y='rating_count_ver')

To effectively analyze discrete variables we need to use a `groupby` query. Pandas [also has great documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html) on this concept so be sure to take a look. Much like with SQL, we can use `groupby` to specify sets of data which we wish to analyze. 

For instance, to analyze the median user rating for each genre, we could create a grouped dataframe by grouping by primary genre. We can then retrieve the median values of each genre easily.

In [ ]:
avg = apps.groupby('prime_genre') # group by primary genre

avg['user_rating'].median() # provide the median user rating for each 

To visualize this on a graph, we could simply create a bar plot. This is a useful visualization for understanding the median user ratings for each genre. Some genres, such as Catalogs, fare poorly.

In [ ]:
avg['user_rating'].median().plot.bar(figsize=(12,6)) 

## *Challenge Question 2 (1 point)*
Create a bar graph which visualizes the mean price for each `content_rating`. Is there a trend? 

In [ ]:
# insert your code here

## *Challenge Question 3 (0.5 points)*
Visualize the sum of the total rating count for each genre. Instead of a bar graph, use a pie chart. If you get stuck, consider [reading the docs](https://pandas.pydata.org/pandas-docs/stable/user_guide/visualization.html#pie-plot) on `plot.pie()`.

In [ ]:
# insert your code here

# Objective 3: Change your plot styles
So far, we have focused on creating various graphs with the data. In addition to creating graphs, we can also change features of these graphs such as their colors, shapes, textures or legend. For instance, if we revisit our scatter plot, we can change the color by specifying `color='DarkGreen'` as an input to the method.

In [ ]:
apps_clean.plot.scatter(x='price', y='rating_count_tot', color='DarkGreen')

You can also specify colors which change with the data variables. The visualization below changes color based on the `user_rating`.

In [ ]:
apps_clean.plot.scatter(x='price', y='rating_count_tot', color=apps_clean['user_rating'])

Finally, you can also change other variables, though these depend on the graph in question. Scatterplots allow you to change based on size, though other graphs can be altered based on variables such as the presence of legends or textures. The graph below changes size depending on the `lang.num`, though this is not terribly informative.

In [ ]:
apps_clean.plot.scatter(x='price', y='rating_count_tot', 
                        c=apps_clean['user_rating'], 
                        s=apps_clean['lang.num'] * 10, figsize=(12,6))

## *Challenge Question 4 (1.5 point)*
Based on what you just learned, create a bar plot which achieves the following:
- Visualizes the 5 most expensive education apps
- Price should be provided on the Y axis
- The application names should be provided on the X axisa
- The graph should have the color `DarkOrange`
- The legend should be removed from the graph

In [ ]:
# insert your code here

# Objective 4: Visualize data in Seaborn
You should now have some familiarity with how to structure and prepare a data frame for visualization. However, there is one remaining problem: these visualizations are ... kind of bland (or... perhaps even downright unattractive)! It is widely recognized among the data science community that the Pandas environment is not well-equipped for visualization. In fact, many data scientists opt to use the R programming language specifically because of its better visualization libraries.

Seaborn was designed to help overcome this problem. As a library built on top of `Matplotlib`, [Seaborn](https://seaborn.pydata.org/index.html) can be used to make nicer-looking visualizations. Let's start by comparing the two. In the cell below, we will render the median values for each `prime_genre` using Matplotlib.

In [ ]:
avg = apps.groupby('prime_genre') # group by primary genre
avg['user_rating'].median().plot.bar(figsize=(12,6)) # visualize the results using matplotlib

This is fine, though it is quite basic and dated. Let's render the same graph in Seaborn. You will notice that the border is removed, and the fonts are modernized. This graph is an improvement!

In [ ]:
import seaborn as sns # import seaborn as an object called sns, like before
sns.set() # sets Seaborn as your default visualization

avg['user_rating'].median().plot.bar(figsize=(12,6)) # call the same visualization

Similarly, with some of our other graphs, we can create something that looks a bit nicer. The graph below is similar to the scatterplot that we saw in Objective 3, visualizing content ratings versus price using the cleaned data. We can change the size of our Seaborn visualizations using the `height` and `aspect` specifications. Try changing these to see what happens.

In [ ]:
sns.relplot(x=apps_clean.price, y=apps_clean.rating_count_tot, 
            hue=apps_clean.cont_rating,
            height=6, aspect=2)

Looking great! Seaborn is a nice tool, and is designed to help create publication quality images based on your data analysis. It comes pre-built with a range of color and design features, which you can read about in the official docs. They also have a bunch of great examples that can be drawn from. Let's try to close this lab by returning to our `highly_rated` data frame which we created in Question 1. As you recall, it tracked a selection of the apps with the highest number of ratings. The bar graph below depicts the ratings based on `rating_count_tot` on the x axis.

In [ ]:
sns.catplot(x="rating_count_tot", y="track_name", data=highly_rated,
            label="Highly rated apps", height=6, aspect=2, kind="bar")

This graph is colorful, but can be improved. Let's sort the results from lowest to greatest. We can also change the color of the bar graph to green by specifying `color="g"`. We can also reduce the size of the graph by changing the `height` and `aspect` variables.

In [ ]:
highly_rated = highly_rated.sort_values(by="rating_count_tot", ascending=True) # sorting these from greatest to least

sns.catplot(x="rating_count_tot", y="track_name", data=highly_rated,
            label="Highly rated apps", color="g", height=4, aspect=1, kind="bar") # changing color to "green"

Finally, it is worth mentioning that you can also specify color palettes that change your graphs to a range of colors, rather than static values. The graph below uses the `mako` pallette. Try changing this to `autumn_r` to see what happens. You can [read more about palettes](https://seaborn.pydata.org/tutorial/color_palettes.html) here.

In [ ]:
sns.catplot(x="rating_count_tot", y="track_name", data=highly_rated,
            label="Highly rated apps", height=4, palette="mako", aspect=2, kind="bar") # try another palette like "autumn_r"

## *Challenge Question 5 (1 point)*
Using Seaborn's `catplot`, visualize the average `user_rating` for each `prime_genre` from the apps data frame. A few things to note:
- There are a lot of categories, so you might struggle to fit them in the default image sizes.
- Consider using `kind="bar"` to tell Seaborn to create a bar chart-- the others might not work here.
- Be default, Seaborn will add lines at the end of your bars. These represent standard deviations, and we will talk more about these next week.
- You will receive full points if you simply create an image that isn't cut off from your notebook though you are welcome to add other features to make the graph look nice if you so choose.

In [ ]:
# insert your code here

# References
Hunter, J. D. (2007). Matplotlib: A 2D graphics environment. IEEE Annals of the History of Computing, 9(03), 90-95.

Waskom, M., Botvinnik, O., Gelbart, M., Ostblom, J., Hobson, P., Lukauskas, S., ... & Brunner, T. (2020). Seaborn: statistical data visualization. Astrophysics Source Code Library, ascl-2012.